In [1]:
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import logging
import json
from pathlib import Path


from wavlm.WavLM import WavLM, WavLMConfig
from hifigan.models import Generator as HiFiGAN
from hifigan.utils import AttrDict
from matcher import ExNOTVC
import torchaudio

import random
import numpy as np

from IPython.display import clear_output
from tqdm.notebook import tqdm as tqdm

DEVICE = 'cuda'

In [2]:
def xnot_vc(pretrained=True, progress=True, prematched=False, device='cuda') -> ExNOTVC:
    """ Load kNN-VC (WavLM encoder and HiFiGAN decoder). Optionally use vocoder trained on `prematched` data. """
    hifigan, hifigan_cfg = hifigan_wavlm(pretrained, progress, prematched, device)
    wavlm = wavlm_large(pretrained, progress, device)
    xnotvc = ExNOTVC(wavlm, hifigan, hifigan_cfg, device)
    return xnotvc


def hifigan_wavlm(pretrained=True, progress=True, prematched=False, device='cuda') -> HiFiGAN:
    """ Load pretrained hifigan trained to vocode wavlm features. Optionally use weights trained on `prematched` data. """
    #cp = Path(__file__).parent.absolute()

    with open('hifigan/config_v1_wavlm.json') as f:
        data = f.read()
    json_config = json.loads(data)
    h = AttrDict(json_config)
    device = torch.device(device)

    generator = HiFiGAN(h).to(device)
    
    if pretrained:
        if prematched:
            url = "https://github.com/bshall/knn-vc/releases/download/v0.1/prematch_g_02500000.pt"
        else:
            print("Загружаем непреметченный")
            url = "https://github.com/bshall/knn-vc/releases/download/v0.1/g_02500000.pt"
        state_dict_g = torch.hub.load_state_dict_from_url(
            url,
            map_location=device,
            progress=progress
        )
        generator.load_state_dict(state_dict_g['generator'])
    generator.eval()
    generator.remove_weight_norm()
    print(f"[HiFiGAN] Generator loaded with {sum([p.numel() for p in generator.parameters()]):,d} parameters.")
    return generator, h


def wavlm_large(pretrained=True, progress=True, device='cuda') -> WavLM:
    """Load the WavLM large checkpoint from the original paper. See https://github.com/microsoft/unilm/tree/master/wavlm for details. """
    if torch.cuda.is_available() == False:
        if str(device) != 'cpu':
            logging.warning(f"Overriding device {device} to cpu since no GPU is available.")
            device = 'cpu'
    checkpoint = torch.hub.load_state_dict_from_url(
        "https://github.com/bshall/knn-vc/releases/download/v0.1/WavLM-Large.pt", 
        map_location=device, 
        progress=progress
    )
    
    cfg = WavLMConfig(checkpoint['cfg'])
    device = torch.device(device)
    model = WavLM(cfg)
    if pretrained:
        model.load_state_dict(checkpoint['model'])
    model = model.to(device)
    model.eval()
    print(f"WavLM-Large loaded with {sum([p.numel() for p in model.parameters()]):,d} parameters.")
    return model


In [3]:

xnotvc = xnot_vc() #загружаем веса для WavLM, HiFi Gan, убираем флажок преметчинга 


/home/wlad/projects/smiles/xnot-vc/.venv/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Загружаем непреметченный
Removing weight norm...
[HiFiGAN] Generator loaded with 16,523,393 parameters.
WavLM-Large loaded with 315,453,120 parameters.


### Пути к аудиодорожкам с женским голосом и мужским

Domain female - домен с женским голосом, Domain male - домен с мужским голосом. Мы хотим из женского голоса сделать мужской

In [4]:
domain_female_path = "data/female_voice_domain.flac"
domain_male_path = "data/male_voice_domain.flac"


### Кодируем аудиодорожки с помощью WavLM 

In [5]:
features_female = xnotvc.get_features(domain_female_path)
features_male = xnotvc.get_features(domain_male_path)

## **KNNVC TEST** (skip)

In [6]:
ref_wav_paths = ["data/male.flac"]
src_wav_path = "data/valid.flac"

In [7]:
query_seq = xnotvc.get_features(src_wav_path)

In [8]:
matching_set = xnotvc.get_matching_set(ref_wav_paths)

In [9]:
out_wav = xnotvc.match(query_seq, matching_set, topk=4)

torch.Size([646, 1024])


In [10]:
import IPython.display as ipd

In [11]:
ipd.Audio(out_wav.numpy(), rate=16000)

In [12]:
torchaudio.save('knnvc_prematched.wav', out_wav[None], 16000)

## **END OF TEST**

WavLM каждые ~20-30 мс звуковой дорожки представляет в виде вектора фичей размерностью 1024

In [13]:
print(features_female.shape) #[длина дорожки делить на ~20-30мс , 1024] = массив из 7971 векторов размерностью 1024

torch.Size([7971, 1024])


In [14]:
print(features_male.shape) #массив из 5473 векторов размерностью 1024

torch.Size([5473, 1024])


### Sampler from voice distributions

Случайным образом выбираем батч векторов фичей

In [15]:
print(features_female.shape)

torch.Size([7971, 1024])


In [16]:
def sample_from_tensor(t, batch_size = 64):
    tensor = t.detach().cpu().numpy()
    batch = np.empty((batch_size, 1024))
    for i in range(batch_size):
        indice = random.randrange(0, tensor.shape[0])
        sample_tensor = tensor[indice]
        batch[i]=sample_tensor
    return batch.astype(np.float32)

In [17]:
sample_from_tensor(features_female).shape

(64, 1024)

In [18]:
sample_from_tensor(features_male).shape

(64, 1024)

### Функция стоимости

In [19]:
def sq_cost(X,Y):
    return (X-Y).square().flatten(start_dim=1).mean(dim=1)

COST = sq_cost

### Define NNs for transport map ***T*** : R^1024 -> R^1024 (generator) and potential ***f*** : R^1024 -> R (discriminator).
Medium size multilayer perceptrons with ReLU.

In [20]:
class NegAbs(nn.Module):
    def __init__(self):
        super(NegAbs, self).__init__()

    def forward(self, input):
        return -torch.abs(input)

T = nn.Sequential(
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024)
).to(DEVICE)

f = nn.Sequential(
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1024),
    nn.ReLU(True),
    nn.Linear(1024, 1),
    NegAbs(),
).to(DEVICE)

def weight_reset(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()


print('T params:', np.sum([np.prod(p.shape) for p in T.parameters()]))
print('f params:', np.sum([np.prod(p.shape) for p in f.parameters()]))

T params: 8396800
f params: 7348225


Оптимизаторы

In [21]:

T_opt = torch.optim.Adam(T.parameters(), lr=1e-4, weight_decay=1e-10)
f_opt = torch.optim.Adam(f.parameters(), lr=1e-4, weight_decay=1e-10)

In [22]:
#Parameters
T_ITERS = 10 # T updates per 1 f update
MAX_ITERS = 15001

to_torch = lambda x: torch.Tensor(x).to(DEVICE)

In [23]:
X = to_torch(sample_from_tensor(features_female))
Y = to_torch(sample_from_tensor(features_male))

In [24]:
sq_cost(X,Y)

tensor([14.3800, 15.0404, 13.0202, 15.1619, 11.8562, 11.8163, 17.1970, 11.9372,
        17.1443, 14.9763, 15.4563, 13.2584, 12.4517, 14.1541, 15.9694, 13.2282,
        11.6751, 15.0750, 18.0247, 12.5239, 14.9292, 14.5387, 15.4272, 17.2312,
        17.6315, 13.8931, 14.2758, 13.0154, 12.2829, 12.3955,  9.7179, 13.2348,
        18.9817, 19.5090, 12.7591, 13.6704, 17.5347, 12.2314, 11.5087, 13.9167,
        17.1061, 15.8710,  7.3214, 14.4751, 13.1643, 16.0897, 20.0949, 10.8914,
        17.3983, 13.2444, 14.6725, 13.2886, 12.6808, 14.5607, 12.8800, 10.3210,
        17.8675, 14.1303,  7.2991, 11.4583,  7.5616, 15.6355, 15.1232,  9.6009],
       device='cuda:0')

## Обучение

Сбрасываем веса

In [25]:
weight_reset(T); weight_reset(f)

Параметр W из алгоритма неполного транспорта

In [26]:
W=16

Алгоритм неполного оптимального транспорта

In [27]:
#XNOT algo

for step in tqdm (range(MAX_ITERS)):
    T_loss_ar = []
    f_loss_ar = []
    #T optimization
    T.train(True); f.eval()
    for t_iter in range(T_ITERS):
        X = to_torch(sample_from_tensor(features_female)) 
        T_loss = COST(X, T(X)).mean() - f(T(X)).mean()
        with torch.no_grad():
            T_loss_ar.append(T_loss.cpu().detach().numpy().mean())
        T_opt.zero_grad(); T_loss.backward(); T_opt.step()

    #f optimization
    T.eval(); f.train(True)
    X, Y = to_torch(sample_from_tensor(features_female)), to_torch(sample_from_tensor(features_male))
    f_loss = f(T(X)).mean() - (W * f(Y)).mean()
    with torch.no_grad():
            f_loss_ar.append(f_loss.cpu().detach().numpy())
    f_opt.zero_grad(); f_loss.backward(); f_opt.step()

    if step % 100 == 0:
        clear_output(wait=True)
        print("Step", step)
        print("T_loss", T_loss_ar)
        print("f_loss", f_loss_ar)




Step 15000
T_loss [np.float32(3.0605264), np.float32(3.1324017), np.float32(2.9184504), np.float32(3.0492105), np.float32(3.1171703), np.float32(3.0271583), np.float32(3.2372642), np.float32(3.0341847), np.float32(2.953469), np.float32(3.028809)]
f_loss [array(0.06441021, dtype=float32)]


Сохраняем модель

In [28]:
torch.save(T.state_dict(), "checkpoints/xnot_vc_15k_W_16.pth")

## Транспорт в новый домен

### Функция, которая заменяет каждый исходный вектор фичей на преобразованный

In [29]:
def xnot_transform(source_tensor):
    with torch.no_grad():
        result = torch.empty(source_tensor.shape[0],1024) #Initialize empty tensor with the same dimensions as an input feature vector
        for i in range(source_tensor.shape[0]):
            result[i] = T(source_tensor[i])

    return result


### Загружаем звуковую дорожку с женским голосом, которой не было в тренировочной выборке и формируем массив из векторов фичей с помощью WavLM

In [30]:
validation_path = "data/valid.flac" #Путь до файла с женским голосом для валидации
features_valid = xnotvc.get_features(validation_path) #Формируем массив из векторов фичей

In [31]:
print(features_valid.shape) #Исходный массив: 646 векторов размерностью 1024


torch.Size([646, 1024])


Формируем преобразованный массив

In [32]:
feautures_transformed = xnot_transform(features_valid) #Преобразованный массив

In [33]:
print(feautures_transformed.shape) #Размерность преобразованного массива должна совпасть с размерностью исходного = 646х1024

torch.Size([646, 1024])


In [34]:
print(feautures_transformed[5])

tensor([ 0.5234, -1.5854,  0.6046,  ...,  2.1043,  0.0491, -3.9298])


## Вокодинг

Применяем метод для вокодинга фичей с помощью HiFi GAN

In [35]:
out_wav = xnotvc.vocode(feautures_transformed[None].to(DEVICE)).cpu().squeeze() #Вокодинг преобразованного с помощью XNOT вектора фичей
        

In [36]:
print(out_wav.shape)

torch.Size([206720])


In [37]:
import IPython.display as ipd

Примеч. У меня внутри ноутбука не выводит звук даже на оригинальных примерах от авторов knn-vc

In [38]:
ipd.Audio(out_wav.numpy(), rate=16000)

Сохраняем файл с аудиодорожкой

In [39]:
torchaudio.save('xnot_vc_15k_W_16.flac', out_wav[None], 16000)